<a href="https://www.kaggle.com/code/ondrejmajor/rohlik-orders-prediction-lstm-w-result-plots?scriptVersionId=190007106" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Rohlík Data Preprocessing
This notebook contains a time series forecasting task on Rohlik dataset. EDA, data preprocessing and forecasting engine is LSTM model with Tensorflow 2 backend. 

<br><b>Approach:</b>
<br>As Rohlík orders dataset contains data from multiple warehouses, thus it contains multiple timelines, the approach is to process and predict each warehouse as a separate timeline. The preprocessing, fitting and evaluation steps are the same for each warehouse. Very wide and shallow LSTM neural net (above 256 units) is used to process the feature rich dataset. Batch size is kept low enough (around 32) to keep the weekly spikes in target sharp.

<br>This approach yields very good MAPE value, which is calculated as a mean of all warehouse timeline results.

<b>This notebook is written with end-to-end mindset as if the models are to be deployed. Models, scalers and training configurations are saved. The test dataset is predicted on with inference style. The predictions can be done offline without the trainig notebook.

In [69]:
#! pip -q install pandas matplotlib numpy seaborn
#! pip install -q scikit-learn
#! pip install -q joblib
#! pip install -q tensorflow

In [70]:
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Flatten, Reshape, LSTM, Dropout, Dense, Bidirectional, BatchNormalization, Input, LayerNormalization, Conv1D, Concatenate, MaxPooling1D, MultiHeadAttention, GlobalAveragePooling1D, Activation, SpatialDropout1D, Lambda
from tensorflow.keras.losses import MeanSquaredError, Huber
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error
import joblib
import os
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler, RobustScaler
from scipy import interpolate
from sklearn.model_selection import KFold

TIME_STEPS = 63
EPOCHS = 25
BATCH_SIZE = 16
LEARNING_RATE = 1e-3

# NUM_FOLDS = 5 # Not implemented

OFF_DAYS = False # Experimental. Fill missing days in timeseries as "offday" and fill features and target with 0s. 
LAG = True # add lag and rolling features
FILL = True # fill nan with zeros before training
SCALER = "standard" # standard best by far...

#WAREHOUSE = ["Prague_1", "Munich_1", "Budapest_1"] # Train only particular warehouse timelines (For testing purposes)
WAREHOUSE = [None]  # Train on all warehouses (whole dataset)


# Temporal features
#LAG_DAYS = [1, 7, 28, 56, 168, 364]  # more mainstream values
#ROLLING_WINDOW = [7, 28, 168, 364]  # more mainstream values
LAG_DAYS = [63, 119, 168, 364]  # this lag bridges the whole test dataset with past data
ROLLING_WINDOW = [63, 168, 364]  # this lag bridges the whole test dataset with past data


pd.set_option('display.max_columns', None)  # see all columns in pandas

train_df_raw = pd.read_csv("/kaggle/input/rohlik-orders-forecasting-challenge/train.csv")
train_calendar_df_raw = pd.read_csv("/kaggle/input/rohlik-orders-forecasting-challenge/train_calendar.csv")
test_df_raw = pd.read_csv("/kaggle/input/rohlik-orders-forecasting-challenge/test.csv")
test_calendar_df_raw = pd.read_csv("/kaggle/input/rohlik-orders-forecasting-challenge/test_calendar.csv")

train_df_raw['date'] = pd.to_datetime(train_df_raw['date'])
train_calendar_df_raw['date'] = pd.to_datetime(train_calendar_df_raw['date'])
test_df_raw['date'] = pd.to_datetime(test_df_raw['date'])
test_calendar_df_raw['date'] = pd.to_datetime(test_calendar_df_raw['date'])

## Optional - Dataset Validation

Evaluate the correctness of the data from multiple sources. Check both .csvs if their values in the overlapping columns match and then merge them into one dataset.
<br>After that we get train_df and test_df dataframes.

In [71]:
def merge_csv(train_df, train_calendar_df):
    # Identify common columns
    common_columns = set(train_df.columns).intersection(set(train_calendar_df.columns))
    print(common_columns)

    # Merg the two datasets on warehouse and date
    merged_df_to_test = pd.merge(train_df, train_calendar_df, on=['date', 'warehouse'], suffixes=('_train', '_calendar'))
    
        # Compare values in common columns
    differences = {}
    for column in common_columns:
        if column not in ['date', 'warehouse']:  # Exclude join keys from comparison
            train_col = f"{column}_train"
            calendar_col = f"{column}_calendar"
            
            # Check if there are any differences, excluding rows where both values are NaN
            diff = merged_df_to_test[(merged_df_to_test[train_col] != merged_df_to_test[calendar_col]) & 
                            (~merged_df_to_test[train_col].isna() | ~merged_df_to_test[calendar_col].isna())]
            if not diff.empty:
                differences[column] = diff[['date', 'warehouse', train_col, calendar_col]]
            else:
                print(f"No differences found in column: {column}")

    # Display only the rows where values were different
    for col, diff in differences.items():
        print(f"\nNot merged. Differences found in column: {col}")
        print(diff)
        return

    if not differences:
        print("\nAll values match across the datasets, datasets merged.")
        return pd.merge(train_df, train_calendar_df, on=list(common_columns))


In [72]:
# Merg the two datasets on 'date' and 'warehouse' to facilitate comparison
train_df_merged = merge_csv(train_df_raw, train_calendar_df_raw)
test_df_merged = merge_csv(test_df_raw, test_calendar_df_raw) 

{'holiday', 'mov_change', 'mini_shutdown', 'precipitation', 'frankfurt_shutdown', 'warehouse', 'school_holidays', 'holiday_name', 'winter_school_holidays', 'blackout', 'shutdown', 'shops_closed', 'date', 'snow'}
No differences found in column: holiday
No differences found in column: mov_change
No differences found in column: mini_shutdown
No differences found in column: precipitation
No differences found in column: frankfurt_shutdown
No differences found in column: school_holidays
No differences found in column: holiday_name
No differences found in column: winter_school_holidays
No differences found in column: blackout
No differences found in column: shutdown
No differences found in column: shops_closed
No differences found in column: snow

All values match across the datasets, datasets merged.
{'holiday', 'warehouse', 'holiday_name', 'winter_school_holidays', 'shops_closed', 'date', 'school_holidays'}
No differences found in column: holiday
No differences found in column: holiday_name

In [73]:
# Sort the datasets by 'date' and 'warehouse' ascendingly

train_df = train_df_merged.sort_values(by=['date', 'warehouse'])
test_df = test_df_merged.sort_values(by=['date', 'warehouse'])

## Drop unnecesary data
There are some features in the training set that are not availiable in the testing set, so we do not use them.
<br>The "orders" column we keep as it is our target.

In [74]:
# get features that are not available in the test dataset
unavailable_features = list(set(train_df.columns).difference(set(test_df.columns)))    
unavailable_features.remove('orders')

print(f"Common features: {unavailable_features}")

Common features: ['mov_change', 'precipitation', 'mini_shutdown', 'frankfurt_shutdown', 'user_activity_1', 'warehouse_limited', 'user_activity_2', 'blackout', 'shutdown', 'snow']


In [75]:
train_df = train_df.drop(columns=unavailable_features)

Columns that have unchanging values or very little movement, sometimes redundant features.
<br> We look as binary features that might be alway 0 or 1.
<br> "school_holidays" and "shops_closed" are redundant in some warehouses
<br> We can drop these columns as they are redundant but in this run we leave it.

In [76]:
# Run the code to identify and drop unchanging columns

for warehouse in train_df['warehouse'].unique():
    unchanging_columns = []
    print(f"\nWarehouse: {warehouse}")
    for feature in train_df.columns:
        if feature in ['warehouse', 'date']:
            continue

        values = train_df[train_df['warehouse'] == warehouse][feature].nunique()
        if values == 1:
            print(f"Feature {feature} has {values} unique values")
            print(f"Value: {train_df[train_df['warehouse'] == warehouse][feature].unique()}")
            if feature not in unchanging_columns:
                unchanging_columns.append(feature)
        if values == 2:
            print(f"Feature {feature} has {values} unique values")
            true_count = sum(train_df[train_df['warehouse'] == warehouse][feature] == 1)
            total_count = len(train_df[train_df['warehouse'] == warehouse])
            print(f"{true_count} items True of {total_count}")

    print(f"    Unchanging columns in {warehouse}: {unchanging_columns}")



Warehouse: Brno_1
Feature holiday has 2 unique values
45 items True of 1193
Feature shops_closed has 2 unique values
20 items True of 1193
Feature winter_school_holidays has 2 unique values
28 items True of 1193
Feature school_holidays has 1 unique values
Value: [0]
    Unchanging columns in Brno_1: ['school_holidays']

Warehouse: Budapest_1
Feature holiday has 2 unique values
9 items True of 1154
Feature shops_closed has 1 unique values
Value: [0]
Feature winter_school_holidays has 2 unique values
6 items True of 1154
Feature school_holidays has 1 unique values
Value: [0]
    Unchanging columns in Budapest_1: ['shops_closed', 'school_holidays']

Warehouse: Prague_1
Feature holiday has 2 unique values
45 items True of 1193
Feature shops_closed has 2 unique values
20 items True of 1193
Feature winter_school_holidays has 2 unique values
84 items True of 1193
Feature school_holidays has 1 unique values
Value: [0]
    Unchanging columns in Prague_1: ['school_holidays']

Warehouse: Prague_

In [77]:
# What is the gap between the date of the last training data and the first test data?
train_max_date = train_df['date'].max()
test_min_date = test_df['date'].min()
print(f"Train max date: {train_max_date}")
print(f"Test min date: {test_min_date}")
print(f"Gap: {test_min_date - train_max_date}")

Train max date: 2024-03-15 00:00:00
Test min date: 2024-03-16 00:00:00
Gap: 1 days 00:00:00


## Experimental - Line-up all warehouse timelines in the same time format.
This is experimental function with a purpose:

-   When we examine the dataset, we can see that some warehouses do not operate at sundays. 
-   Some warehouses are missing a lot of days rom first months as they were just setting up oin new markets.
-   There are missing days on other warehouses sometimes.
-   <b>This function introduces</b> a new feature <b>"off_day"</b>, which is 1 whenever we have missing day in the dataset.
- Currently diabled, uncomment last line bellow to experiment.

In [78]:
def process_warehouses(train_df, handle_off_days='interpolate'):
    """
    Process each warehouse and handle off days based on the specified method.
    
    Parameters:
    train_df (DataFrame): The input DataFrame containing warehouse data.
    handle_off_days (str): The method to handle off days, either 'interpolate' or 'zeros'.
    
    Returns:
    DataFrame: The processed DataFrame with all warehouses combined.
    """
    combined_df_list = []
    start_dates_info = []

    for warehouse in train_df['warehouse'].unique():
        wh = train_df[train_df['warehouse'] == warehouse]
        
        # Fix specific warehouses with unconisitent data at the start
        if warehouse == 'Frankfurt_1':
            first_month_end = wh['date'].min() + pd.DateOffset(days=4)
            wh = wh[wh['date'] > first_month_end]
        if warehouse == 'Munich_1':
            first_month_end = wh['date'].min() + pd.DateOffset(weeks=5)
            wh = wh[wh['date'] > first_month_end]
            
        wh.set_index('date', inplace=True)
        date_range = pd.date_range(start=wh.index.min(), end=wh.index.max(), freq='D')
        wh_reindexed = wh.reindex(date_range).reset_index()
        wh_reindexed.rename(columns={'index': 'date'}, inplace=True)
        
        # Add off_day column
        wh_reindexed['off_day'] = wh_reindexed['orders'].isna().astype(int)
        
        # Here we have to fill in the missing features for the off days, mostly 0s here, we do not count holidays, etc. for missing days for now.
        wh_reindexed['warehouse'] = warehouse
        wh_reindexed['holiday_name'] = wh_reindexed['holiday_name'].fillna('')
        wh_reindexed['holiday'] = wh_reindexed['holiday'].fillna(0)
        wh_reindexed['shops_closed'] = wh_reindexed['shops_closed'].fillna(0)
        wh_reindexed['winter_school_holidays'] = wh_reindexed['winter_school_holidays'].fillna(0)
        wh_reindexed['school_holidays'] = wh_reindexed['school_holidays'].fillna(0)
        
        if handle_off_days == 'interpolate':
            wh_reindexed['orders'] = wh_reindexed['orders'].interpolate(method='linear')
        elif handle_off_days == 'zeros':
            wh_reindexed['orders'] = wh_reindexed['orders'].fillna(0.0)
        
        # the 'id' column is for the test dataset mostly
        wh_reindexed['id'] = wh_reindexed.apply(lambda row: f"{row['warehouse']}_{row['date'].strftime('%Y-%m-%d')}", axis=1)
        wh_reindexed['day_of_week'] = wh_reindexed['date'].dt.dayofweek + 1  # Monday=1, Sunday=7
        
        # Cut off data to start on the earliest Monday - all warehouses start on timeline on Monday, so they hawe weekly overlap.
        start_date = wh_reindexed['date'].min()
        end_date = wh_reindexed['date'].max()
        start_monday = start_date - pd.Timedelta(days=start_date.weekday())
        end_sunday = end_date + pd.Timedelta(days=(6 - end_date.weekday()))
        wh_reindexed = wh_reindexed[(wh_reindexed['date'] >= start_monday) & (wh_reindexed['date'] <= end_sunday)]
        
        # get info about the start date 
        start_dates_info.append(f"{warehouse}: Start Date: {start_monday.strftime('%Y-%m-%d')} (Day {start_monday.weekday() + 1})")

        # get all new dataframes into a list
        combined_df_list.append(wh_reindexed)
        
        # Plot
        plt.figure(figsize=(20, 10))
        plt.plot(wh_reindexed['date'], wh_reindexed['orders'], marker='x', linestyle='-', markersize=3, linewidth=0.5, label='Orders')

        # start of the first week and end of the last week
        plt.axvline(x=start_monday, color='r', linestyle='--', linewidth=0.5, label='Start of first week')
        plt.axvline(x=end_sunday, color='g', linestyle='--', linewidth=0.5, label='End of last week')
        
        # Highlight the off days
        off_days = wh_reindexed[wh_reindexed['off_day'] == 1]
        plt.scatter(off_days['date'], off_days['orders'], color='red', s=15, label='Off days', marker='o')
        
        # Mark the weekdays on off days
        for idx, row in wh_reindexed.iterrows():
            if row['off_day'] == 1:
                plt.text(row['date'], row['orders'], str(row['day_of_week']), color='black', ha='center', va='bottom', fontsize=10)

        plt.title(f"Volume for warehouse {warehouse}")
        plt.xlabel('Date')
        plt.ylabel('Orders')
        plt.legend()
        plt.xticks(rotation=45)
        plt.show()

        # Drop day_of_week column , we will be adding it later as a feature
        wh_reindexed.drop(columns=['day_of_week'], inplace=True)

    # Print start dates for each warehouse
    print("Start Dates for Each Warehouse:")
    for info in start_dates_info:
        print(info)

    # Combine all warehouses into one DataFrame
    train_df_reindexed = pd.concat(combined_df_list, ignore_index=True)
    
    return train_df_reindexed

# Uncomment to use this function
if OFF_DAYS:
    train_df = process_warehouses(train_df, handle_off_days='zeros') # zero or interpolate - zero makes more sense but still confuses the model durin training. Interpolate is purely experimental.

We are merging train and test datasets and process it as one dataset, so we can apply temporal features on both datasets. We will get lag days on the test dataset. Later we split dataset back to train and test for training and prediction.

In [ ]:
# merge the two datasets
train_df = pd.concat([train_df, test_df], axis=0)

In our test dataset is not a lot of holidays that had inpact on "orders" in previous years so we are not going to use it. 
<br>It could be used with one-hot encoding and a bit experimentuing.

In [ ]:
train_df = train_df.drop(columns=['holiday_name'])

In [ ]:
train_df = train_df.sort_values(by=['date', 'warehouse'], ignore_index=True)

In [ ]:
train_df.head(7)

In [ ]:
# only get data of one warehouse for the dataset - for testing purposes
if WAREHOUSE != [None]:
    train_df = train_df[train_df['warehouse'].isin(WAREHOUSE)]
    print("ran")

## Produce date features
This function processes the date column and produces multiple time features from it.

In [ ]:
from math import pi
def process_date(df):
    
    df['date'] = pd.to_datetime(df['date'])
    
    # Extract date-related features
    df['quarter'] = df['date'].dt.quarter
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['day_of_week'] = df['date'].dt.dayofweek
    df['day_of_year'] = df['date'].dt.dayofyear
    
    df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)

    
    # Calculate days since the start date
    df['days_since_start'] = (df['date'] - df['date'].min()).dt.days
    
    
    # Create cyclic features
    def cyclical_encode(df, col, max_val):
        df[f'{col}_sin'] = np.sin(2 * np.pi * df[col] / max_val)
        df[f'{col}_cos'] = np.cos(2 * np.pi * df[col] / max_val)
    
    cyclical_encode(df, 'month', 12)
    cyclical_encode(df, 'day_of_week', 7)
    #cyclical_encode(df, 'day_of_year', 365)
    #cyclical_encode(df, 'day', 30)

    cat_features = ["day_of_week", 'quarter', 'month']
    bin_features = ['is_weekend']
    return df, cat_features, bin_features

In [ ]:
# apply process_date function and save the categorical and binary features for later use.

train_df, categorical_features, binary_features=process_date(train_df)

if OFF_DAYS:
    binary_features = binary_features + ['holiday', 'shops_closed', 'winter_school_holidays', 'school_holidays', 'off_day']
else:
    binary_features = binary_features + ['holiday', 'shops_closed', 'winter_school_holidays', 'school_holidays']

We keep track of important parameters and settings during the training phase, as it is good habbit when later deploying machine learning models for inference.
<br>Here we are tracking features that will be passed through the scalers.

In [ ]:
x_scaler_features = list(set(train_df.columns) - set(categorical_features) - set(binary_features) - {'warehouse', 'orders', 'id', 'date'})
print(x_scaler_features)
print(categorical_features)

In [ ]:
if "off_day" in binary_features:
    train_df['off_day'] = train_df['off_day'].fillna(0)

    orders = train_df["orders"]
    train_df = train_df.fillna(0)
    train_df["orders"] = orders

In [ ]:
# Review dataset before proecssing
train_df.head(7)

## Correlation Matrix

In [ ]:
train_df_corr = train_df.drop(columns=['warehouse', "id"]).corr()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(train_df_corr, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix')
plt.show()

# Print the correlation with 'orders'
print(train_df_corr['orders'].sort_values(ascending=False))

## Produce temporal features
Create lag and rolling window features.

In [ ]:
def create_lag_features(df, col, lag_days):
    df = df.copy()
    lag_features = []
    for lag in lag_days:
        feature_name = f'{col}_lag_{lag}'
        df[feature_name] = df[col].shift(lag)
        lag_features.append(feature_name)
    return df, lag_features

def create_rolling_features(df, col, windows):
    df = df.copy()
    window_features = []
    for window in windows:
        mean_feature = f'{col}_rolling_mean_{window}'
        std_feature = f'{col}_rolling_std_{window}'
        
        df[mean_feature] = df[col].rolling(window, min_periods=1).mean()
        df[std_feature] = df[col].rolling(window, min_periods=1).std()
        
        window_features.extend([mean_feature, std_feature])
    return df, window_features

In [ ]:
def preprocess_data(df, cat_features, x_scaler_features, binary_features, redundant_features=[], inference=False, fill_method='ffill', lag = True):
    warehouse = df['warehouse'].iloc[0]
    redundant_df = df[redundant_features]
    df = df.drop(columns=redundant_features)
    
    if inference is False and lag is True:
        # Create lag and rolling features
        df, lag_features = create_lag_features(df, 'orders', LAG_DAYS)
        df, window_features = create_rolling_features(df, 'orders', ROLLING_WINDOW)
        
        x_scaler_features = x_scaler_features + lag_features + window_features
        
        # Handle NaN values in lag and rolling features
        if fill_method == 'ffill':
            df[lag_features + window_features] = df[lag_features + window_features].ffill().bfill()
            orders = df["orders"]
            df = df.ffill().bfill()
            df["orders"] = orders
            
        elif fill_method == 'zero':
            df[lag_features + window_features] = df[lag_features + window_features].fillna(0)
            orders = df["orders"]
            df = df.fillna(0)
            df["orders"] = orders
    
    # One-hot encoding for categorical features
    if not inference:
        encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
        encoded_features = encoder.fit_transform(df[cat_features])
        joblib.dump(encoder, f'onehot_scaler_{warehouse}.joblib')
    else:
        encoder = joblib.load(f'onehot_scaler_{warehouse}.joblib')
        encoded_features = encoder.transform(df[cat_features])

    encoded_feature_names = encoder.get_feature_names_out(cat_features)
    encoded_df = pd.DataFrame(encoded_features, columns=encoded_feature_names, index=df.index)

    # Prepare features for scaling
    if inference is False:
        X = df.drop(columns=['orders'])
        y = df['orders']
    else:
        X = df

    # Initialize and fit/transform scalers
    if not inference:
        if SCALER == "minmax":
            x_scaler = MinMaxScaler(feature_range=(0, 1))
            y_scaler = MinMaxScaler(feature_range=(0, 1))
        elif SCALER == "robust":
            x_scaler = RobustScaler()
            y_scaler = RobustScaler()
        else:
            x_scaler = StandardScaler()
            y_scaler = StandardScaler()
        X_scaled = x_scaler.fit_transform(X[x_scaler_features])
        y_scaled = y_scaler.fit_transform(y.values.reshape(-1, 1))
        joblib.dump(x_scaler, f'x_scaler_{warehouse}.joblib')
        joblib.dump(y_scaler, f'y_scaler_{warehouse}.joblib')
    else:
        x_scaler = joblib.load(f'x_scaler_{warehouse}.joblib')
        X_scaled = x_scaler.transform(X[x_scaler_features])
        #y_scaler = joblib.load(f'y_scaler_{warehouse}.joblib')
        #y_scaled = y_scaler.transform(y.values.reshape(-1, 1))

    # Create DataFrame with scaled features
    X_scaled_df = pd.DataFrame(X_scaled, columns=x_scaler_features, index=df.index)

    # Combine all features
    final_df = pd.concat([
        X_scaled_df,  # Scaled numerical features
        encoded_df,   # One-hot encoded categorical features
        df[binary_features],  # Binary features (unchanged)
        redundant_df  # Add redundant features back for the test data export
    ], axis=1)
    
    if inference:
        print(final_df.shape)
        return final_df
    else:
        print(final_df.shape, y_scaled.shape)
        return final_df, y_scaled

## Data Preprocessing Function
This is preprocessing function that applies:
<br>lag features
<br>one-hot encoding of categorical features
<br>feature-target split
<br>feature, target scaling
<br>return the final dataframe

## Outliers Reduction Function
Function to fix outliers - orders that are very high or very low. 
<br>Plot the outliers.
<br>Settings of the z-score and window that is outliers calculated with.

In [ ]:
if not OFF_DAYS:
    def z_score_outlier_detection(data, window=14, threshold=3):
        data = data.sort_index()
        rolling_mean = data.rolling(window=window, center=True, min_periods=1).mean()
        rolling_std = data.rolling(window=window, center=True, min_periods=1).std()
        z_scores = (data - rolling_mean) / rolling_std
        outliers = (np.abs(z_scores) > threshold) & (data != 0)
        lower_bound = rolling_mean - (threshold * rolling_std)
        upper_bound = rolling_mean + (threshold * rolling_std)
        return outliers, lower_bound, upper_bound


    def weighted_trend_interpolation(data, outliers, window_size=10, weight=0.5):
        fixed_data = data.copy()
        for idx in np.where(outliers)[0]:
            start_idx = max(0, idx - window_size)
            end_idx = min(len(data), idx + window_size + 1)

            prev_values = data.iloc[start_idx:idx].dropna()
            next_values = data.iloc[idx + 1:end_idx].dropna()

            #print(f"Previous values: {prev_values}\n")
            #print(f"Next values: {next_values}\n")

            if len(prev_values) == 0:
                correction = next_values.mean()
            elif len(next_values) == 0:
                correction = prev_values.mean()
            else:
                mean_prev = prev_values.mean()
                mean_next = next_values.mean()

                if mean_next > mean_prev:
                    trend = 'up'
                else:
                    trend = 'down'

                # Calculate new value based on trend and retain some of the original intensity
                if trend == 'up':
                    correction = mean_prev + (next_values.mean() - mean_prev) / 2
                else:
                    correction = mean_prev - (mean_prev - next_values.mean()) / 2

            if np.isnan(correction):  # Handle cases where both prev_values and next_values are NaN
                correction = data.iloc[idx]

            fixed_data.iloc[idx] = weight * data.iloc[idx] + (1 - weight) * correction

            #print(f"Outlier at index {idx}: {data.iloc[idx]} -> {fixed_data.iloc[idx]}")
        return fixed_data


    def plot_outlier_comparison(df, window=28, z_threshold=3, window_size=10, weight=0.5):
        warehouse_data = df['orders']

        z_outliers, z_lower, z_upper = z_score_outlier_detection(warehouse_data, window, z_threshold)
        fixed_warehouse_data = weighted_trend_interpolation(warehouse_data, z_outliers, window_size, weight)

        # Plot the original data and outliers
        plt.figure(figsize=(20, 10))
        plt.plot(warehouse_data.index, warehouse_data, label='Orders', alpha=0.7)
        plt.fill_between(warehouse_data.index, z_lower, z_upper, color='green', alpha=0.1, label='Z-Score Bounds')
        plt.scatter(warehouse_data.index[z_outliers], warehouse_data[z_outliers], color='blue', label='Z-Score Outliers', marker='s')

        # Plot fixed outliers
        plt.scatter(warehouse_data.index[z_outliers], fixed_warehouse_data[z_outliers], color='green', label='Fixed Outliers', marker='x')
        plt.title(f'{df["warehouse"].iloc[0]} - Outlier Comparison')
        plt.xlabel('Date')
        plt.ylabel('Orders')
        plt.legend()
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

        print(f"\n{df['warehouse'].iloc[0]}:")
        print(f"Z-Score Outliers: {sum(z_outliers)}")

        # Update the DataFrame with fixed values
        df.loc[:, 'orders'] = fixed_warehouse_data

        return df



else:
    def z_score_outlier_detection(data, window=14, threshold=3):
        # Exclude rows where 'off_day' == 1
        mask = data['off_day'] != 1
        data_filtered = data[mask].copy()
        data_filtered['orders'] = data_filtered['orders'].sort_index()

        rolling_mean = data_filtered['orders'].rolling(window=window, center=True, min_periods=1).mean()
        rolling_std = data_filtered['orders'].rolling(window=window, center=True, min_periods=1).std()
        z_scores = (data_filtered['orders'] - rolling_mean) / rolling_std
        outliers_filtered = (np.abs(z_scores) > threshold) & (data_filtered['orders'] != 0)
        lower_bound_filtered = rolling_mean - (threshold * rolling_std)
        upper_bound_filtered = rolling_mean + (threshold * rolling_std)

        # Create full-length arrays with NaN for excluded indices
        outliers = pd.Series(np.nan, index=data.index)
        lower_bound = pd.Series(np.nan, index=data.index)
        upper_bound = pd.Series(np.nan, index=data.index)

        outliers[mask] = outliers_filtered
        lower_bound[mask] = lower_bound_filtered
        upper_bound[mask] = upper_bound_filtered

        return outliers, lower_bound, upper_bound

    def weighted_trend_interpolation(data, outliers, window_size=10, weight=0.5):
        fixed_data = data['orders'].copy()
        for idx in np.where(outliers)[0]:
            start_idx = max(0, idx - window_size)
            end_idx = min(len(data), idx + window_size + 1)

            prev_values = data['orders'].iloc[start_idx:idx].dropna()
            next_values = data['orders'].iloc[idx + 1:end_idx].dropna()

            if len(prev_values) == 0:
                correction = next_values.mean()
            elif len(next_values) == 0:
                correction = prev_values.mean()
            else:
                mean_prev = prev_values.mean()
                mean_next = next_values.mean()

                if mean_next > mean_prev:
                    trend = 'up'
                else:
                    trend = 'down'

                # Calculate new value based on trend and retain some of the original intensity
                if trend == 'up':
                    correction = mean_prev + (next_values.mean() - mean_prev) / 2
                else:
                    correction = mean_prev - (mean_prev - next_values.mean()) / 2

            if np.isnan(correction):  # Handle cases where both prev_values and next_values are NaN
                correction = data['orders'].iloc[idx]

            fixed_data.iloc[idx] = weight * data['orders'].iloc[idx] + (1 - weight) * correction

        return fixed_data

    def plot_outlier_comparison(df, window=28, z_threshold=3, window_size=10, weight=0.5):
        warehouse_data = df[['orders', 'off_day']]
        non_off_day_data = warehouse_data[warehouse_data['off_day'] != 1]

        z_outliers, z_lower, z_upper = z_score_outlier_detection(warehouse_data, window, z_threshold)
        fixed_warehouse_data = weighted_trend_interpolation(warehouse_data, z_outliers, window_size, weight)

        # Plot the original data and outliers
        plt.figure(figsize=(20, 10))
        plt.plot(non_off_day_data.index, non_off_day_data['orders'], label='Orders', alpha=0.7)
        plt.fill_between(warehouse_data.index, z_lower, z_upper, color='green', alpha=0.1, label='Z-Score Bounds')
        plt.scatter(warehouse_data.index[z_outliers.dropna().index], warehouse_data['orders'][z_outliers.dropna().index], color='blue', label='Z-Score Outliers', marker='s')

        # Plot fixed outliers
        plt.scatter(warehouse_data.index[z_outliers.dropna().index], fixed_warehouse_data[z_outliers.dropna().index], color='green', label='Fixed Outliers', marker='x')
        plt.title(f'{df["warehouse"].iloc[0]} - Outlier Comparison')
        plt.xlabel('Date')
        plt.ylabel('Orders')
        plt.legend()
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

        print(f"\n{df['warehouse'].iloc[0]}:")
        print(f"Z-Score Outliers: {sum(z_outliers.dropna())}")

        # Update the DataFrame with fixed values
        df.loc[:, 'orders'] = fixed_warehouse_data

        return df

In [ ]:
train_df.isna().sum()

In [ ]:
print(test_df['date'].min())
print(test_df['date'].max())

## Preprocess Data for Each Warehouse, Data Visualisations
Put all above together in a loop that produces:

- Reduces outliers
- Train dataframe dictionary for every warehouse a dataframe
- Test dataframe with lag features and scaled 
- Extra dates from train data added to the test dataset to create LSTM sequences from later.


In [ ]:
from datetime import timedelta


features = [f'orders_lag_{lag}' for lag in LAG_DAYS]
features2 = [f'orders_rolling_mean_{window}' for window in ROLLING_WINDOW]

warehouses = train_df['warehouse'].unique()

# Create dataframes for each warehouse
warehouse_dfs = {wh: train_df[train_df['warehouse'] == wh].copy() for wh in warehouses}


processed_train_data_dict = {}
processed_test_data = []

for warehouse, df in warehouse_dfs.items():
    print(f"Processing warehouse: {warehouse}")
    
    
    
    df = df.sort_values(by=['date'])
    
    # Get the date range for test dataset.
    test_min_date = test_df['date'].min()
    test_max_date = test_df['date'].max()
    
    # Get the first date of the test data adding extra dates to it for the sequences for LSTM.
    start_date = test_min_date - (2*timedelta(days=TIME_STEPS))
    
    # Get indices for the specified date range for the test dataset.
    selected_indices = df[(df['date'] >= start_date) & (df['date'] <= test_max_date)].index
    # Indicies to drop from train data, tat only include the real test data from the real test dataset.
    test_indices = df[(df['date'] >= test_min_date) & (df['date'] <= test_max_date)].index
    
    # Plot outlier setup
    df = plot_outlier_comparison(df, window=70, z_threshold=2.8, weight=0.5)

    # Preprocess data
    X_processed, y_processed = preprocess_data(df, 
                                               categorical_features, 
                                               x_scaler_features, 
                                               binary_features, 
                                               redundant_features=['warehouse', 'id', 'date'], 
                                               inference=False, 
                                               fill_method='ffill',
                                               lag=LAG)
    
    
    
    
    test_data = X_processed.loc[selected_indices]
    
    selected_positions = X_processed.index.get_indexer(selected_indices)
    test_positions = X_processed.index.get_indexer(test_indices)
    
    X_processed = X_processed.drop(test_indices)
    y_processed = np.delete(y_processed, test_positions)
    
    processed_test_data.append(test_data)
    processed_train_data_dict[warehouse] = (X_processed, y_processed)
    
    print(f"Processed:\ntrain data shape: {X_processed.shape}, {y_processed.shape}")
    print("test data shape:", test_data.shape)
    print(f"y nans: {sum(np.isnan(y_processed))}")
    print("\n")

# create test dataframe for the test data for each warehouse
processed_test_df = pd.concat(processed_test_data)


# Export Processed Test Data

In [ ]:
if FILL:
    processed_test_df = processed_test_df.fillna(0)
    processed_test_df.tail()

In [ ]:
sum(processed_test_df.isna().sum())

In [ ]:
print(f"Processed test dataset length: {len(processed_test_df)}")
print(f"Raw test dataset length: {len(test_df_raw)}")
# Save the test dataset for future use
processed_test_df.to_csv('/kaggle/working/test_proc_mt.csv')

### Plot the test data
Visualise the training dataset with new features for every warehouse timeline

In [ ]:
if LAG:
    features = [f'orders_lag_{lag}' for lag in LAG_DAYS]
    features2 = [f'orders_rolling_mean_{window}' for window in ROLLING_WINDOW]

    def is_weekend(date):
        return date.weekday() >= 5

    # Plot the processed test data:

    for wh in processed_test_df['warehouse'].unique():
        wh_df = processed_test_df[processed_test_df['warehouse'] == wh]
        plt.figure(figsize=(20, 10))

        plt.plot(wh_df["date"], wh_df[features[0]], label=features[0])
        plt.plot(wh_df["date"], wh_df[features[1]], label=features[1])
        plt.plot(wh_df["date"], wh_df[features[2]], label=features[2])
        plt.plot(wh_df["date"], wh_df[features[3]], label=features[3])
        plt.plot(wh_df["date"], wh_df[features2[0]], label=features2[0])
        plt.plot(wh_df["date"], wh_df[features2[1]], label=features2[1])
        plt.plot(wh_df["date"], wh_df[features2[2]], label=features2[2])

        # Shade weekends
        wh_df['is_weekend'] = wh_df['date'].apply(is_weekend)
        weekends = wh_df[wh_df['is_weekend']]
        for i, row in weekends.iterrows():
            plt.axvspan(row['date'], row['date'] + pd.Timedelta(days=1), color='gray', alpha=0.3)

        plt.title(f'{wh} - Processed Test Data')
        plt.xlabel('Date')
        plt.ylabel('Orders')
        plt.legend()
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

### Plot LAG features
Visualise LAG features of train and test back to back with the real orders

In [ ]:
if LAG:
    feature = f'orders_lag_{LAG_DAYS[2]}'  # third lag feature for test data

    # Loop through the warehouse dataframes in the dictionary
    for warehouse in processed_train_data_dict.keys():
        plt.figure(figsize=(15, 5))

        # Get train data for the warehouse
        train_orders = processed_train_data_dict[warehouse][1]  # X_processed
        train_data = processed_train_data_dict[warehouse][0]  # X_processed

        # Get test data for the warehouse
        test_data = processed_test_df[processed_test_df['warehouse'] == warehouse]

        # Plot the orders and the feature
        plt.plot(train_data.index, train_orders, label='Train Orders', color='green')
        plt.plot(train_data.index, train_data[feature], label=f'Train {feature}', color='blue')
        plt.plot(test_data.index, test_data[feature], label=f'Test {feature}', color='red')
        
        plt.title(f'Train and Test for {warehouse} Warehouse - {feature}')
        plt.xlabel('Date')
        plt.ylabel('Orders')
        plt.legend()
        plt.show()

In [ ]:
for warehouse, (X_processed, y_processed) in processed_train_data_dict.items():
    print(f"Warehouse: {warehouse}")
    print("Start Date:", X_processed["date"].min())
    print("End Date:", X_processed["date"].max())
    print()

In [ ]:
# Review dataset before training
print(len(processed_train_data_dict[warehouse][0]))
processed_train_data_dict[warehouse][0].head()


# Training the Model
- Create time series sequences
- Create LSTM neural net model
- Plot loss function history from traing
- Training loop

In [ ]:
def create_sequences(X, y=None, time_steps=10):
    sequences = []
    targets = []

    if y is not None:
        # Training mode
        if len(X) > time_steps:
            generator = TimeseriesGenerator(X.values, 
                                            y,
                                            length=time_steps, 
                                            batch_size=1)
            
            for i in range(len(generator)):
                x, y = generator[i]
                sequences.append(x)
                targets.append(y)
        
        X_seq = np.array(sequences)
        y_seq = np.array(targets)
        X_seq = np.squeeze(X_seq, axis=1)
        y_seq = np.squeeze(y_seq, axis=1)
        
        print(f"Training mode - final shape: X: {X_seq.shape}, y: {y_seq.shape}")
        
        return X_seq, y_seq
    
    else:
        # Inference mode
        for i in range(len(X) - time_steps + 1):
            seq = X.iloc[i:(i + time_steps)].values
            sequences.append(seq)
        
        X_seq = np.array(sequences)
        
        print(f"Inference mode - final shape: X: {X_seq.shape}")
        
        return X_seq

## Model Architecture
Model from experimenting, simpler gets more constant resulets

In [ ]:
from tensorflow.keras.regularizers import l2


def create_model(input_shape, lr=1e-3, warehouse="None"):

    model = Sequential()
    model.add(Input(shape=input_shape))
    
    if warehouse in ["Brno_1", "Prague_1", "Prague_2", "Prague_3"]:

        #model.add(Conv1D(filters=32, kernel_size=3, activation='relu', padding='same', kernel_regularizer=l2()))
        #model.add(BatchNormalization())
        #model.add(Dropout(0.4))
        
        #model.add(Bidirectional(LSTM(units=64, activation='relu', return_sequences=True)))
        #model.add(BatchNormalization())
        #model.add(Dropout(0.2))
        #model.add(LSTM(128, return_sequences=True))
        model.add(LSTM(1024))
        #model.add(BatchNormalization())
        #model.add(Dropout(0.3))
        
        #model.add(Dense(16))
        #model.add(BatchNormalization())
        #model.add(Dropout(0.1))
    
        
    elif warehouse in ["Munich_1", "Frankfurt_1"]:

        #model.add(Conv1D(filters=16, kernel_size=3, activation='relu', padding='same', kernel_regularizer=l2()))
        #model.add(BatchNormalization())
        #model.add(Dropout(0.3))
        
        #model.add(Bidirectional(LSTM(units=64, activation='relu', return_sequences=True)))
        #model.add(BatchNormalization())
        #model.add(Dropout(0.4))
        model.add(LSTM(512, kernel_regularizer=l2()))
        #model.add(BatchNormalization())
        #model.add(Dropout(0.3))
        
        #model.add(Dense(16))
        #model.add(BatchNormalization())
        #model.add(Dropout(0.3))
    
    elif warehouse in ["Budapest_1"]:
        #model.add(Conv1D(filters=16, kernel_size=2))
        #model.add(BatchNormalization())
        #model.add(Dropout(0.3))
        
        model.add(LSTM(1024, kernel_regularizer=l2()))
        #model.add(BatchNormalization())
        #model.add(Dropout(0.3))
        
        
        
    else:
        print("No model selected, unknown warehouse timeline.")
        return

    model.add(Dense(1))

    #model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=lr))
    model.compile(loss=MeanSquaredError(), optimizer=tf.keras.optimizers.RMSprop(learning_rate=lr, rho=0.9))
                  
    return model


In [ ]:
def plot_loss(history, warehouse):
    plt.figure(figsize=(12, 8))

    # training and validation loss
    plt.plot(history.history['loss'], label='Training Loss', color='blue', linewidth=2)
    plt.plot(history.history['val_loss'], label='Validation Loss', color='orange', linewidth=2)
    
    # minimum validation loss
    min_val_loss = min(history.history['val_loss'])
    min_val_loss_epoch = history.history['val_loss'].index(min_val_loss)
    plt.axvline(min_val_loss_epoch, linestyle='--', color='gray', linewidth=1)
    plt.text(min_val_loss_epoch, min_val_loss, f'Min Val Loss: {min_val_loss:.4f}', 
             verticalalignment='bottom', horizontalalignment='right', color='gray', fontsize=10)
    
    plt.title(f'Training and Validation Loss for Warehouse: {warehouse}', fontsize=16)
    plt.xlabel('Epoch', fontsize=14)
    plt.ylabel('Loss', fontsize=14)
    plt.legend(fontsize=12)
    plt.grid(True)

    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.tight_layout()
    
    plt.savefig(f'training_validation_loss_{warehouse}.png', dpi=300)
    
    plt.show()

In [ ]:
#print nan values for every warehouse in train dataset dict:
for warehouse in processed_train_data_dict.keys():
    print(f"Nan X values for {warehouse}: {processed_train_data_dict[warehouse][0].isna().sum().sum()}")
    print(f"nan y values for {warehouse}: {sum(np.isnan(processed_train_data_dict[warehouse][1]))}")
    
    if FILL:
        feat, targ = processed_train_data_dict[warehouse]

        feat = feat.fillna(0)
        targ = np.nan_to_num(targ, nan=0.0)
        processed_train_data_dict[warehouse] = (feat, targ)

if FILL:
    print("\n After FILL:")
    for warehouse in processed_train_data_dict.keys():
        print(f"Nan X values for {warehouse}: {processed_train_data_dict[warehouse][0].isna().sum().sum()}")
        print(f"nan y values for {warehouse}: {sum(np.isnan(processed_train_data_dict[warehouse][1]))}")


## Training Loop
Training loop, create sequences, train-test split, create model, use callbacks for trainig, fit the model.

In [ ]:
#### Training loop:

val_data = {}
training_features = {}

for warehouse, (X_train_scaled, y_train_scaled) in processed_train_data_dict.items():
    print(f"Training model for warehouse: {warehouse}")

    X_train_scaled = X_train_scaled.drop(columns=['id', 'warehouse', 'date'])
    
    training_features[warehouse] = X_train_scaled.columns.tolist()

    # Create sequences
    X_seq, y_seq = create_sequences(X_train_scaled, y_train_scaled, time_steps=TIME_STEPS)
    
    # train-test split
    X_train, X_val, y_train, y_val = train_test_split(X_seq, y_seq, test_size=0.2, shuffle=True)
    
    # Create, compile the model
    model = create_model(input_shape=(X_train.shape[1], X_train.shape[2]), lr=LEARNING_RATE, warehouse=warehouse)
    
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
        ModelCheckpoint(f'model_{warehouse}.keras', save_best_only=True, monitor='val_loss', mode='min', verbose=1),
        #ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=1e-6)
    ]
    
    # Save the test split of the dataset for each warehouse for future evaluation.
    val_data[warehouse] = (X_val, y_val)
    
    history = model.fit(
        X_train, y_train, 
        epochs=EPOCHS, 
        batch_size=BATCH_SIZE, 
        #validation_split=0.2, 
        validation_data=(X_val, y_val), 
        callbacks=callbacks,
        verbose=1
    )
    
    plot_loss(history, warehouse)

In [ ]:
X_train_scaled.describe()

Save all scaler features into a single json scaler config file and save the scaler config so it can be loded for inference.

In [ ]:
scaler_config = {
    'bin_features': binary_features,
    'cat_features': categorical_features,
    'scaler_features': x_scaler_features,
    'training_features': training_features,
}

joblib.dump(scaler_config, 'scaler_config_mt.joblib')

# MAPE Evaluation

## Visualize Predictions on the Training Data

In [ ]:
#tf.keras.config.enable_unsafe_deserialization()

# Load scalers

# Initialize a list to store MAPE scores
mape_scores = []

# Loop through each warehouse and calculate MAPE
for warehouse, (X_train_scaled, y_train_scaled) in processed_train_data_dict.items():
    print(f"Predicting for warehouse: {warehouse}")
    y_scaler = joblib.load(f'y_scaler_{warehouse}.joblib')
    X_train_scaled = X_train_scaled.drop(columns=['id', 'warehouse', 'date'])
    # Create sequences
    X_seq, y_seq = create_sequences(X_train_scaled, y_train_scaled, time_steps=TIME_STEPS)

    # Load the model
    model = tf.keras.models.load_model(f'model_{warehouse}.keras')

    # Predict
    predictions = model.predict(X_seq)
    predictions = y_scaler.inverse_transform(predictions)
    y_seq = y_seq.reshape(-1, 1)
    y_seq_unscaled = y_scaler.inverse_transform(y_seq)

    # Calculate MAPE
    mape = mean_absolute_percentage_error(y_seq_unscaled, predictions) * 100
    mape_scores.append(mape)

    # Visualise the predictions
    plt.figure(figsize=(15, 5))
    plt.plot(y_seq_unscaled, label='True', color='black')
    plt.plot(predictions, label='Predicted', color='red')
    plt.plot(y_seq_unscaled - predictions, label='Difference', color='blue', linestyle='--')
    plt.title(f'Predictions vs True for warehouse: {warehouse}')
    plt.xlabel('Date')
    plt.ylabel('Orders')
    plt.legend()
    plt.grid(True)
    plt.show()

    print(f'\nMean Absolute Percentage Error on training data for {warehouse}: {mape:.4f}%')

# Print MAPE scores
print("\nMAPE training scores: ")
for i, score in enumerate(mape_scores):
    print(f"{warehouses[i]}: {score:.4f}%")

print(f"\nMean training MAPE: {np.mean(mape_scores):.4f}%")

## Visualize Predictions on the Validation Data

In [ ]:
mape_scores = []

for warehouse in warehouses:
    print(f"Predicting for warehouse: {warehouse}")
    y_scaler = joblib.load(f'y_scaler_{warehouse}.joblib')
    # Load the validation data for the current warehouse
    X_val, y_val = val_data[warehouse]
    
    # Load the model
    model = tf.keras.models.load_model(f'model_{warehouse}.keras')
        # Predict
    predictions = model.predict(X_val)
    predictions = y_scaler.inverse_transform(predictions)
    y_val = y_val.reshape(-1, 1)
    y_val_unscaled = y_scaler.inverse_transform(y_val)
    
    # Calculate MAPE
    mape = mean_absolute_percentage_error(y_val_unscaled, predictions) * 100
    mape_scores.append(mape)
    
    # Visualise the predictions
    plt.figure(figsize=(15, 5))
    plt.plot(y_val_unscaled, label='True', color='black')
    plt.plot(predictions, label='Predicted', color='red')
    plt.plot(y_val_unscaled - predictions, label='Difference', color='blue', linestyle='--')
    plt.title(f'Predictions vs True for warehouse: {warehouse}')
    plt.xlabel('Sample')
    plt.ylabel('Orders')
    plt.legend()
    plt.show()
    
    print(f'\nMean Absolute Percentage Error on validation data for {warehouse}: {mape:.4f}%')

print("\nMAPE validation scores: ")
for i, score in enumerate(mape_scores):
    print(f"{warehouses[i]}: {score:.4f}%")

print(f"\nMean validation MAPE: {np.mean(mape_scores):.4f}%")


## Inference and Visualisation on the TEST Data:

Here we deploy our trained models and run inference on the test data from the challenge. The test data are already preprocessed from the previous step, so we only load them and create LSTM sequences, then predict. If we wanted to take raw test data, we would only need to utilize the "process_data" function from the above section first.

This sequence function is not using the TimeseriesGenerator, because I could not figure out how to use it so it would utilize all the test data rows, there were always leftovers.
This function is creating sequences for each row and when it does not have enough data, it is padding the sequence with the first row of the sequence.

Function that that predicts the test data on multiple timelines - splitting the data for each warehouse, using multiple models.

In [ ]:
def predict_multiple(data, time_steps, warehouses_trained, training_features):
    predictions = []
    submission_data = []

    for warehouse in data['warehouse'].unique():
        if warehouse not in warehouses_trained:
            print(f"No model found for warehouse: {warehouse}")
            warehouse_predictions = [np.nan] * len(data[data['warehouse'] == warehouse])
            predictions.extend(warehouse_predictions)
            continue
        model_data = warehouses_trained[warehouse]
        model = model_data['model']
        y_scaler = model_data['y_scaler']
        wh_data = data[data['warehouse'] == warehouse].copy()
        wh_data = wh_data.reindex(columns=training_features[warehouse])
    

        sequences = create_sequences(wh_data, None, time_steps)
        print(f"{warehouse}: ")
        print(len(sequences))
        # Predict on all sequences
        preds = model.predict(sequences)
        preds_rescaled = y_scaler.inverse_transform(preds)

        # Align predictions with original data
        wh_predictions = np.full(len(wh_data), np.nan)

        # Calculate the number of predictions
        num_predictions = len(preds_rescaled)

        # Assign predictions, accounting for the offset due to sequence creation
        wh_predictions[-num_predictions:] = preds_rescaled.flatten()
        predictions.extend(wh_predictions)

        # Prepare submission data
        wh_submission_data = data.loc[data['warehouse'] == warehouse].copy()

        wh_submission_data['predicted_orders'] = wh_predictions

        submission_data.append(wh_submission_data)

    submission_df = pd.concat(submission_data)

    submission_df.dropna(inplace=True)

    return np.array(predictions), submission_df

Load the saved models and scalers for each warehouse. This is optional and a conscious choise to use and is a part of end-to-end approach with potential deployment in mind.

In [ ]:
def load_models_and_scalers(warehouses):
    warehouse_models_and_scalers = {}

    for wh in warehouses:
        try:
            model = tf.keras.models.load_model(f'model_{wh}.keras')
            x_scaler = joblib.load(f'x_scaler_{wh}.joblib')
            y_scaler = joblib.load(f'y_scaler_{wh}.joblib')
            encoder = joblib.load(f'onehot_scaler_{wh}.joblib')
            
            warehouse_models_and_scalers[wh] = {
                'model': model,
                'x_scaler': x_scaler,
                'y_scaler': y_scaler,
                'encoder': encoder
            }
        except Exception as e:
            print(f"Error loading model or scaler for warehouse {wh}: {str(e)}")

    return warehouse_models_and_scalers

Here we load the scaler configuration and other metadata from the training stage

In [ ]:
scaler_config = joblib.load('scaler_config_mt.joblib') # load the scaler config

x_scaler_features = scaler_config['scaler_features']   # standard scaler features in the right order that the training dataset was scaled on
categorical_features = scaler_config['cat_features']    # categorical features
binary_features = scaler_config['bin_features']    # binary features are unpluged before scaling and then plugged back in after scaling
training_features = scaler_config['training_features']    # training features dictionary
    
print(f"cat_features: {categorical_features}")
print(f"binary_features: {binary_features}")
print(f"scaler_columns: {len(x_scaler_features)}")

 Load the saved preprocessed training data

In [ ]:
dataframe_raw = pd.read_csv("/kaggle/working/test_proc_mt.csv")
dataframe_raw['date'] = pd.to_datetime(dataframe_raw['date'])
dataframe = dataframe_raw.sort_values(by=['date', 'warehouse'])
print(f"Rows: {len(dataframe)}")

In [ ]:
dataframe.head()

## Load models and predict the test dataset

In [ ]:
warehouse_lengths = test_df.groupby('warehouse').size()

for warehouse, length in warehouse_lengths.items():
    print(f"{warehouse}: {length}")

In [ ]:
warehouse_models = load_models_and_scalers(warehouses)
predictions, submission_df = predict_multiple(dataframe, TIME_STEPS, warehouse_models, training_features)

In [ ]:
submission_df = submission_df.rename(columns={"predicted_orders": "orders"})

submission_df = submission_df[submission_df['date'].isin(test_df_raw['date'])]

submission_export = submission_df[["id", "orders"]]

In [ ]:
print(len(processed_test_df))
print(len(submission_df))
print(sum(submission_df.isna().sum()))

In [ ]:
submission_df.head()

In [ ]:
submission_export.to_csv("/kaggle/working/submission.csv", index=False)

## Visualise the Test Data Predictions
Visualise the predictions for 2 months of the test data with the actual data from previous year to see how the predictions fit.

In [ ]:
# Plot predictions for each warehouse
train_df_raw = pd.read_csv("/kaggle/input/rohlik-orders-forecasting-challenge/train.csv")
train_df_raw['date'] = pd.to_datetime(train_df_raw['date'])

last_year_data = train_df_raw[train_df_raw['date'].dt.year > 2022]

for wh in warehouses:

    wh_df_pred = submission_df[submission_df['warehouse'] == wh]
    wh_df_pred = wh_df_pred.sort_values(by=['date'])
    wh_df_last_year = last_year_data[last_year_data['warehouse'] == wh]
    wh_df_last_year = wh_df_last_year.sort_values(by=['date'])
    
    # Plot data (training data from 2023)

    plt.figure(figsize=(12, 6))
    
    plt.plot(wh_df_pred.date, wh_df_pred['orders'], label='Predicted',  linestyle='--', color='red')

    plt.plot(wh_df_last_year.date, wh_df_last_year['orders'], label='Actual', color='blue')

    plt.title(f'Predicted vs Actual Orders for {wh} in 2023-2024')
    plt.xlabel('Date')
    plt.ylabel('Orders')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.savefig(f'prediction_plot_{wh}.png')
    plt.show()
    plt.close()

print("Predictions complete. Submission file and plots created.")

In [ ]:
city = "Prague_1"
submission_single_df = submission_df[submission_df['date'].isin(test_df_raw['date'])]
submission_single_df = submission_single_df[submission_single_df.warehouse == city]
submission_single_df = submission_single_df[["id", "orders", "date", "warehouse"]]
submission_single_df.to_csv(f"/kaggle/working/submission_{city}.csv", index=False)



In [ ]:
submission_single_df.head()

In [ ]:

train_df_raw = pd.read_csv("/kaggle/input/rohlik-orders-forecasting-challenge/train.csv")
train_df_raw['date'] = pd.to_datetime(train_df_raw['date'])

#last_year_data = train_df_raw[train_df_raw['date'].dt.year > 2022]


wh_df_pred = submission_single_df[submission_single_df['warehouse'] == city]
wh_df_pred = wh_df_pred.sort_values(by=['date'])
wh_df_last_year = last_year_data[last_year_data['warehouse'] == city]
wh_df_last_year = wh_df_last_year.sort_values(by=['date'])


plt.figure(figsize=(12, 6))

plt.plot(wh_df_pred.date, wh_df_pred['orders'], label='Predicted',  linestyle='--', color='red')

plt.plot(wh_df_last_year.date, wh_df_last_year['orders'], label='Actual', color='blue')

plt.title(f'Predicted vs Actual Orders for {city}')
plt.xlabel('Date')
plt.ylabel('Orders')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.savefig(f'prediction_plot_{wh}.png')
plt.show()
plt.close()

# Testing ground:

In [ ]:
x_val, y_val = val_data["Prague_1"]
x_val.shape

In [ ]:
x_val_test = dataframe[dataframe.warehouse == "Prague_1"]
x_val_test_reindex = x_val_test.reindex(columns=training_features["Prague_1"])
x_test_sequences = create_sequences(x_val_test_reindex, None, TIME_STEPS)
x_test_sequences.shape

In [ ]:
array1_flat = x_val.reshape(-1, x_val.shape[-1])
array2_flat = x_test_sequences.reshape(-1, x_test_sequences.shape[-1])

In [ ]:
df1_val = pd.DataFrame(array1_flat)
df2_sub = pd.DataFrame(array2_flat)

In [ ]:
df2_sub.head()

In [ ]:
print(X_train_scaled.shape)
X_train_scaled.head()

In [ ]:

df1_val.head()

In [ ]:
print(processed_train_data_dict["Prague_1"][0].shape)
processed_train_data_dict["Prague_1"][0].head()


In [ ]:
print(submission_df.shape)
submission_df.head()

## 